In [2]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import col
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.mllib.evaluation import RankingMetrics
import plotly.express as px
import random 
import time
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

# for NDCG
from pyspark.sql import Window
from pyspark.sql.functions import col
from pyspark.sql.functions import expr
import pyspark.sql.functions as F

read ratings

In [4]:
line_count = len(open("/Users/jessie/Desktop/Personalization/Project2/yelp_dataset/review.json").readlines())
user_ids, business_ids, stars, dates = [], [], [], []
with open("/Users/jessie/Desktop/Personalization/Project2/yelp_dataset/review.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        user_ids += [blob["user_id"]]
        business_ids += [blob["business_id"]]
        stars += [blob["stars"]]
        dates += [blob["date"]]
ratings = pd.DataFrame(
{"user_id": user_ids, "business_id": business_ids, "rating": stars, "date": dates}
)

100%|██████████| 6685900/6685900 [01:05<00:00, 102349.21it/s]


In [5]:
# save and load the dataframe
#ratings.to_pickle('ratings.pkl')
ratings = pd.read_pickle('ratings.pkl')
ratings.head()

,user_id,business_id,rating,date
0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-05-07 04:34:36
1,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,2017-01-14 21:30:33
2,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,2016-11-09 20:09:03
3,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,2018-01-09 20:56:38
4,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,2018-01-30 23:07:38


read business

In [ ]:
line_count = len(open("/Users/jessie/Desktop/Personalization/Project2/yelp_dataset/business.json").readlines())
business_ids, names, addresses, cities, states, postal_codes, latitudes, longitudes, stars, review_counts,\
is_open, attributes, categories, hours = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
with open("/Users/jessie/Desktop/Personalization/Project2/yelp_dataset/business.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        business_ids += [blob["business_id"]]
        names += [blob["name"]]
        addresses += [blob["address"]]
        cities += [blob["city"]]
        states += [blob["state"]]
        postal_codes += [blob["postal_code"]]
        latitudes += [blob["latitude"]]
        longitudes += [blob["longitude"]]
        stars += [blob["stars"]]
        review_counts += [blob["review_count"]]
        is_open += [blob["is_open"]]
        attributes += [blob["attributes"]]
        categories += [blob["categories"]]
        hours += [blob["hours"]]
        
business = pd.DataFrame(
{"business_id": business_ids, "name": names, "address": addresses, "city": cities, "state": states, \
"postal_code": postal_codes, "latitude": latitudes, "longitude": longitudes, "stars":stars, \
"review_count":review_counts, "is_open":is_open, "attributes":attributes, "categories":categories, "hours":hours}
)

In [ ]:
business.head()

In [4]:
# add integer user ID and item ID, ALS can only read interger user ID and item ID
ratings_1=ratings
ratings_1['user_code'] = ratings_1.user_id.astype('category').cat.codes
ratings_1['item_code'] = ratings_1.business_id.astype('category').cat.codes

# create diectionary (mapping)
user_dict = dict(zip(ratings_1.user_code, ratings_1.user_id))
item_dict = dict(zip(ratings_1.item_code, ratings_1.business_id))

In [6]:
ratings_1.head()

,user_id,business_id,rating,date,user_code,item_code
0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-05-07 04:34:36,1158189,176747
1,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,2017-01-14 21:30:33,1599679,74082
2,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,2016-11-09 20:09:03,1307908,100925
3,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,2018-01-09 20:56:38,1064624,140760
4,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,2018-01-30 23:07:38,1455311,117663


In [7]:
# select active users
# select the id of active users
user_counts = ratings["user_id"].value_counts()
active_users = user_counts.loc[user_counts >= 5].index.tolist()
active_ratings=ratings_1.loc[ratings_1['user_id'].isin(active_users)].sort_values(by=['user_id', 'date'])

In [8]:
# select the most recent rating as test, the rest as training
test = active_ratings.groupby('user_id').tail(1)
training = active_ratings.drop(test.index)
print("number of rows in training: ",len(training))
print("number of rows in test: ",len(test))

number of rows in training:  4252142
number of rows in test:  286130


In [9]:
# randomly select 50% training as training, otherwise the dataset is too large
# training_sample = training.sample(frac=0.5, random_state=12345)

In [10]:
# convert to spark dataframe for ALS fitting, drop unrelated columns
test_spark = spark.createDataFrame(test[['user_code','item_code','rating']])
training_spark = spark.createDataFrame(training[['user_code','item_code','rating']])

## Model fitting - Matrix factorization

In [9]:
# fit matrix factorization model
#als = ALS(userCol="user_code", itemCol="item_code", ratingCol="rating",coldStartStrategy="drop")
#model = als.fit(training_spark)

In [10]:
# Build the ALS model
als = ALS(userCol="user_code", itemCol="item_code", ratingCol="rating",coldStartStrategy="drop")

# Set parameter map and metric
paramGrid = (ParamGridBuilder()
              .addGrid(als.rank, [10,15])    # number of latent factors in the model
              .addGrid(als.maxIter, [10,15]) # maximum number of iterations to run
              .addGrid(als.regParam, [0.01,0.1,0.5]).build())   # regularization parameter in ALS
rmse_evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating")

# 5-Fold Cross Validation (preserve each submodel)
crossval = CrossValidator(estimator=als,
                           estimatorParamMaps=paramGrid,
                           evaluator=rmse_evaluator,
                           numFolds=5)

# Fit the model by training dataset.
cvModel = crossval.fit(training_spark)

In [11]:
# Get the best model from cross validation
best_model = cvModel.bestModel

# Print evaluation metrics and model parameters
print ("**Best Model**")
print ("Rank: ", best_model.rank)
print ("MaxIter: ",best_model._java_obj.parent().getMaxIter())
print ("RegParam: ",best_model._java_obj.parent().getRegParam())

**Best Model**
Rank:  10
MaxIter:  15
RegParam:  0.5


In [13]:
# save or load the model
cvModel.save('MFcvModel')
#from pyspark.ml.tuning import CrossValidatorModel
#cvModel = CrossValidatorModel.load('MFcvModel')

In [14]:
# Evaluate the model by computing the RMSE on the test data
predictions = cvModel.transform(test_spark)
rmse=rmse_evaluator.evaluate(predictions)
print ("test RMSE = ", rmse)

test RMSE =  1.4370287007673046


In [15]:
print("number of rows dropped for cold start: ",len(test)-predictions.count())

number of rows dropped for cold start:  2919


In [15]:
# save and load predictions
#predictions.write.save('predictions', format='parquet')
predictions = spark.read.load("predictions") 

### group by popularity of item

In [19]:
# group the items into popular (>=300 ratings), moderate (10-300 ratings) and unpopular (<10 ratings)
item_counts = active_ratings["item_code"].value_counts()
popular_items = item_counts.loc[item_counts >= 300].index.tolist()
moderate_items = item_counts.loc[(item_counts < 300) & (item_counts >= 10)].index.tolist()
unpopular_items = item_counts.loc[item_counts < 10].index.tolist()

In [20]:
rmse_popular=rmse_evaluator.evaluate(predictions.filter(predictions.item_code.isin(popular_items)))
rmse_moderate=rmse_evaluator.evaluate(predictions.filter(predictions.item_code.isin(moderate_items)))
rmse_unpopular=rmse_evaluator.evaluate(predictions.filter(predictions.item_code.isin(unpopular_items)))
print("rmse of popular items: " , rmse_popular)
print("rmse of moderate items: " , rmse_moderate)
print("rmse of unpopular items: " , rmse_unpopular)

rmse of popular items:  1.315975732947149
rmse of moderate items:  1.431661381240479
rmse of unpopular items:  1.650259185689958


In [22]:
# count the number of items in each group
print("number of popular items: " , len(popular_items))
print("number of moderate items: " , len(moderate_items))
print("number of unpopular items: " , len(unpopular_items))

number of popular items:  1844
number of moderate items:  70549
number of unpopular items:  113330


### group by activeness of users

In [16]:
# group the users into active (>=20 ratings), moderate (10-20 ratings) and inactive (<10 ratings)
user_counts = active_ratings["user_code"].value_counts()
active_users = user_counts.loc[user_counts >= 20].index.tolist()
moderate_users = user_counts.loc[(user_counts < 20) & (user_counts >= 10)].index.tolist()
inactive_users = user_counts.loc[user_counts < 10].index.tolist()

In [18]:
rmse_activeU=rmse_evaluator.evaluate(predictions.filter(predictions.user_code.isin(active_users)))
rmse_moderateU=rmse_evaluator.evaluate(predictions.filter(predictions.user_code.isin(moderate_users)))
rmse_inactiveU=rmse_evaluator.evaluate(predictions.filter(predictions.user_code.isin(inactive_users)))
print("rmse of active users: " , rmse_activeU)
print("rmse of moderate users: " , rmse_moderateU)
print("rmse of inactive users: " , rmse_inactiveU)

rmse of active users:  1.3399711012473254
rmse of moderate users:  1.4070938148225576
rmse of inactive users:  1.4775870789989252


In [23]:
# count the number of users in each group
print("number of active users: " , len(active_users))
print("number of moderate users: " , len(moderate_users))
print("number of inactive users: " , len(inactive_users))

number of active users:  47944
number of moderate users:  74880
number of inactive users:  163306
